## Data Preparation
We'll start by importing the necessary libraries and preparing the CIFAR-10 dataset.

In [ ]:
!pip install torch torchvision


  Using cached torchvision-0.18.1-cp310-cp310-win_amd64.whl (1.2 MB)
  Using cached torch-2.3.1-cp310-cp310-win_amd64.whl (159.8 MB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached tbb-2021.13.0-py3-none-win_amd64.whl (286 kB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl (3.5 MB)
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2


ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from torch.utils.data import DataLoader, Subset
import numpy as np

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transformations for the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


In [ ]:
# Download CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

In [ ]:
# Function to split dataset into shards and slices
def split_dataset(dataset, num_shards, num_slices):
    shard_size = len(dataset) // num_shards
    shards = [Subset(dataset, range(i * shard_size, (i + 1) * shard_size)) for i in range(num_shards)]
    slices = []
    for shard in shards:
        slice_size = len(shard) // num_slices
        slices.append([Subset(shard, range(j * slice_size, (j + 1) * slice_size)) for j in range(num_slices)])
    return slices

In [ ]:
# Example split
num_shards = 5
num_slices = 5
slices = split_dataset(train_dataset, num_shards, num_slices)
